# Topic Modelling

## Libraries

In [100]:
import pandas as pd
import numpy as np
import nltk
import re
import gensim
#import pyLDAvis.gensim
import warnings
import csv
from datetime import datetime as dt
from sklearn.externals import joblib

from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')
# nltk.download()

## Options

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))       # Changing the cell widths

pd.options.display.max_rows = 30                                            # Setting the max number of rows
pd.options.display.max_columns = 50                                         # Setting the max number of columns

#pyLDAvis.enable_notebook()

## Variables

In [ ]:
path = 'data/'    # Data Directory 
out = 'Outputs/'  # Output Directory
msg = 'messages.csv'                       # Input Dataset

sample_size = 5000
topics = 10

## Import

In [45]:
df_msg_in = pd.read_csv(path + msg)
df_msg_en = df_msg_in[(df_msg_in['language'] == 'EN')] 

## Functions

In [80]:
class Message(object):
    def __init__(self, thread_id, date_time, message_id, user_id, language, msg_type, msg_body):
        self.thread_id = thread_id
        self.date_time = date_time
        self.message_id = message_id
        self.user_id = user_id
        self.language = language
        self.msg_type = msg_type
        self.msg_body = msg_body
        
    
class User(object):
    def __init__(self):
        pass

def loadData(f):
    users = {}
    messages = []
    with open(f, "r") as data:
        # "thread_id","date_time","message_id","user_id","language","type","body"
        # datetime format: 2015-02-09 14:27:05
        reader = csv.DictReader(data)
        for row in reader:
            user_id = row["user_id"]
            message = Message(row["thread_id"],
                              dt.strptime(row["date_time"], "%Y-%m-%d %H:%M:%S"),
                              row["message_id"],
                              row["user_id"],
                              row["language"],
                              row["type"],
                              row["body"])
            if user_id not in users:
                users[user_id] = []
            users[user_id].append(message)
            messages.append(message)
    return users, messages

def getMessageGroups(messages, grouper):
    groupedMessages = {}
    for message in messages:
        messageGroup = getattr(message, grouper)
        if messageGroup not in groupedMessages:
            groupedMessages[messageGroup] = []
        groupedMessages[messageGroup].append(message)
    return groupedMessages
        

def cleaner(row):
    '''Function to clean the text data and prep for further analysis'''
    stops = set(stopwords.words("english"))     # Creating a set of Stopwords
    p_stemmer = PorterStemmer()                 # Creating the stemmer model
    text = row['body'].lower()                   # Converts to lower case
    text = re.sub("[^a-zA-Z]"," ",text)          # Removes punctuation
    text = re.sub("cyclist","cycl",text)         # Manual intervention for 'cyclist'
    text = text.split()                          # Splits the data into individual words 
    text = [w for w in text if not w in stops]   # Removes stopwords
    text = [p_stemmer.stem(i) for i in text]     # Stemming (reducing words to their root)
    return text


def messages_vectorizer(messages):
    '''Function to take a message object and convert it to a list of terms'''
    stops = set(stopwords.words("english"))     # Creating a set of Stopwords
    p_stemmer = PorterStemmer()                 # Creating the stemmer model
    text = ''
    for m in messages:
        text = text + ' ' + m.msg_body.lower()          # Converts to lower case
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.split()                          # Splits the data into individual words 
    text = [w for w in text if not w in stops]   # Removes stopwords
    text = [p_stemmer.stem(i) for i in text]     # Stemming (reducing words to their root)
    return text

def model(data):
    data_dict = corpora.Dictionary(data)                       # Creates an id <-> term dictionary
    data_corpus = [data_dict.doc2bow(text) for text in data]     # convert tokenized documents into a document-term matrix
    data_model = gensim.models.ldamodel.LdaModel(data_corpus, 
                                                   num_topics=topics, 
                                                   id2word = data_dict,
                                                   passes=20)        #  generate LDA model

    #data_vis = pyLDAvis.gensim.prepare(data_model, data_corpus, data_dict)        # Visualise LDA Model
    #pyLDAvis.save_html(data=data_vis,
    #                    fileobj=out + 'Data_vis.html')
    #data_vis
    return data_model, data_corpus, data_dict


In [49]:
users, messages = loadData(path + msg)
grouped_messages =  getMessageGroups(messages, 'user_id')

In [54]:
grouped_messages.keys()[1]

'58146'

In [ ]:
message_count_dict={}

for k,v in grouped_messages.items():
   message_count_dict[k] = len(v) 



import operator
sorted_message_count_dict = sorted(message_count_dict.items(), key=operator.itemgetter(1))
sorted_message_count_dict.reverse()

sorted_message_count_dict

In [81]:
grouped_user_data =[]

min_messages = 3
max_messages = 300

grouped_messages_by_user = grouped_messages
for k in grouped_messages.keys():
    v = grouped_messages[k]
    if (len(v)<max_messages and len(v)>min_messages):
        grouped_user_data.append(messages_vectorizer(v))

        


In [82]:
users_topic_model = model(grouped_user_data)

In [102]:
the_model = users_topic_model[0]
the_corpus = users_topic_model[1]
word_dict = users_topic_model[2]

the_model.get_document_topics(the_corpus[0])

joblib.dump(users_topic_model, '../Outputs/users_topic_model.pkl' ) 



for k,v in the_model.get_topic_terms(0,topn=25):
    print word_dict[k]

use
plant
soil
control
water
crop
feed
diseas
fertil
cow
maiz
best
farm
anim
manur
make
caus
weed
spray
appli
well
good
product
dri
seed


In [46]:
df_msg_en['Clean'] = df_msg_en.apply(cleaner,axis=1)

df_msg_en_q = df_msg_en[(df_msg_en['type'] == 'Q')]
df_msg_en_a = df_msg_en[(df_msg_en['type'] == 'A')]

li_msg_en_q = df_msg_en_q['Clean'].tolist()
li_msg_en_a = df_msg_en_a['Clean'].tolist()

/Users/dyaaalbakour/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
sample = li_msg_en_q[:sample_size]
data_model, data_corpus = model(sample)

In [ ]:
data_model.get_document_topics(data_corpus[0])